In [1]:
from Pipeline_preprocess_dataset_hc_new import Create_Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

# Load Iniziale Dataset

In [2]:
# df = pd.read_csv('../data/Complete/DATASET_COMPLETO_HC.csv')
df = pd.read_csv(r'C:\Users\AlessandroKuz\Desktop\ITS\PW01 - Project Work\ULTIMATE WORK\Clean data and scripts\HC_TRAIN_DF.csv')

In [3]:
df["LABEL"] = df["LABEL"].map({"NEG": 0, "POS": 1, "VUS": 2})

In [4]:
df['LABEL'].value_counts()

LABEL
0    19566
2      144
1       11
Name: count, dtype: int64

In [5]:
masks_label = df['LABEL'] == 2
pre_train_df, predict_df = df[~masks_label], df[masks_label]

In [6]:
X, y = pre_train_df.drop(['LABEL'], axis=1), pre_train_df['LABEL']
X_pred, y_pred = predict_df.drop(['LABEL'], axis=1), predict_df['LABEL']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [8]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=21, stratify=y_test)

# Pipeline Preprocessing

In [9]:
my_pipi = Create_Pipeline()

In [10]:
X_train = my_pipi.fit_transform(X_train)

KeyError: 'STORIA_FAMILIARE_POS'

In [ ]:
# colonne_fam = []
# for col in X_train.columns:
#     if col.startswith('SEDE_TUMORE_FAMILIARITA'):
#         colonne_fam.append(col)
#     elif col.startswith('AOP'):
#         colonne_fam.append(col)
# # colonne_fam

for col in X_train.columns:
    if col.startswith('AOP'):
        X_test[col] = 0
        X_val[col] = 0
    elif col.startswith('SEDE_TUMORE_FAMILIARITA'):
        X_test[col] = 0
        X_val[col] = 0

In [ ]:
# for column in colonne_fam:
#     print(X_train[column].value_counts())

In [ ]:
# for col in colonne_fam:
#     if col not in X_test.columns:
#         X_test[col] = 0

In [ ]:
# X_test.columns

In [ ]:
X_test = my_pipi.transform(X_test)

In [ ]:
X_val = my_pipi.transform(X_val)

# Selezione Modello

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

random_forest = RandomForestClassifier()
logistic_regression = LogisticRegression()
svc = SVC()

# Splitting Normale

In [ ]:
random_forest_test = random_forest.fit(X_train, y_train)

In [ ]:
random_forest_test.score(X_val, y_val)

In [ ]:
predictions = random_forest_test.predict(X_val)
# getting unique values of the numpy array
# np.unique(predictions, return_counts=True)
# getting the row where the value is 1
np.where(predictions == 1)

In [ ]:
X_pred.columns

In [ ]:
for col in X_train.columns:
    if col.startswith('AOP'):
        X_pred[col] = 0
    elif col.startswith('SEDE_TUMORE_FAMILIARITA'):
        X_pred[col] = 0

In [ ]:
X_pred.columns

In [ ]:
X_pred = my_pipi.transform(X_pred)

In [ ]:
predictions = random_forest_test.predict(X_pred)

np.unique(predictions, return_counts=True)

# Splitting con K-fold

In [ ]:
train_fold = StratifiedKFold(n_splits=10, random_state=21, shuffle=True)
val_fold = StratifiedKFold(n_splits=10, random_state=21, shuffle=True)

In [ ]:
for i, (train_index, test_index) in enumerate(train_fold.split(X, X['STRATIFY_COL'])):
    X_train_temp, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train_temp, y_test = y.iloc[train_index], y.iloc[test_index]

    for j, (train_index, val_index) in enumerate(val_fold.split(X_train_temp, X_train_temp['STRATIFY_COL'])):
        X_train, X_val = X_train_temp.iloc[train_index], X_train_temp.iloc[val_index]
        y_train, y_val = y_train_temp.iloc[train_index], y_train_temp.iloc[val_index]

        # Model training and evaluation
        random_forest = RandomForestClassifier()
        random_forest_test = random_forest.fit(X_train, y_train)
        score = random_forest_test.score(X_val, y_val)

        # Model prediction
        predictions = random_forest_test.predict(X_pred)

        # Model evaluation
        np.unique(predictions, return_counts=True)

        # Save model
        import pickle


        